In [1]:
import VGG as vgg
import torch
from HSI_class import HSI
import os
import numpy as np

PATH = r"C:\Users\Asus TUF\Documents\code\TA\models\model_800train\model_20250228_175649_1"


print("creating model...")
saved_model = vgg.VGG16_HSI()
saved_model.load_state_dict(torch.load(PATH))
print("model created")

creating model...


c:\Users\Asus TUF\Documents\code\TA\myenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Asus TUF\Documents\code\TA\myenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\Asus TUF\AppData\Local\Temp\ipykernel_5112\730607048.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/py

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Asus TUF\\Documents\\code\\TA\\models\\model_800train\\model_20250228_175649_1'

In [ ]:
dataset_path = "Hyperspectral oil spill detection datasets"

dataset = []

i = 0
for filename in os.listdir(dataset_path):
    if i > 3:
        break
    file_path = os.path.join(dataset_path, filename)
    if os.path.isfile(file_path):  # Check if it's a file
        print(f"Processing file: {file_path}")
        hsi = HSI(file_path)
        dataset.append(hsi)
    i += 1

In [ ]:
import zeroPadding
hsi_test = dataset[2]

test_img = hsi_test.img
test_gt = hsi_test.gt

patch_size = 9
half_patch = patch_size // 2

height = test_img.shape[0]
width = test_img.shape[1]

matrix=zeroPadding.zeroPadding_3D(test_img,half_patch) #add 0 in every side of the data
print(f"img shape: {test_img.shape}")
print(f"img shape after padding {matrix.shape}")
print(f"number of pixel {width * height}")

In [ ]:
def predict(input):

    with torch.no_grad():  # Disable gradients for inference
        output = saved_model(input)

    # Convert logits to class label
    predicted_class = torch.argmax(output, dim=1).item()
    confidence = torch.nn.functional.softmax(output, dim=1)[0, predicted_class].item()

    return predicted_class, confidence

In [ ]:
print(test_gt.shape)

indices0 = np.argwhere(test_gt == 0)
indices1 = np.argwhere(test_gt == 1)

print(indices0.shape)
print(indices1.shape)

num_samples = 100

random_indices0 = indices0[np.random.choice(len(indices0), num_samples, replace=False)]
random_indices1 = indices1[np.random.choice(len(indices1), num_samples, replace=False)]

test_indices = np.vstack((random_indices0, random_indices1))

print(test_indices.shape)

In [ ]:
total = 0
correct = 0

i = 0
for indice in test_indices:
    x_pos = indice[0]
    y_pos = indice[1]

    true_label = test_gt[x_pos][y_pos]

    selected_rows = matrix[range(x_pos,x_pos+2*half_patch+1), :]
    testing_patch = selected_rows[:, range(y_pos, y_pos+2*half_patch+1)]
    
    # print(i)
    # print(testing_patch[half_patch][half_patch])
    # print(test_img[x_pos][y_pos])

    testing_patch = torch.tensor(testing_patch)
    testing_patch = testing_patch.to(torch.float32)
    testing_patch = testing_patch.unsqueeze(0)
    testing_patch = testing_patch.permute(0, 3, 1, 2)

    prediction, confidence = predict(testing_patch)

    print(f"{i+1}: prediction = {prediction}, confidence: {confidence}, expected: {true_label}")
    
    if(prediction == true_label):
        correct += 1

    total += 1
    i += 1

print(f"skor: {correct}/{total}")